In [149]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

com_df = pd.read_csv('cite반영유사도데이터_수정.csv')

In [155]:
com_df = com_df[com_df['Filtered']=='o']
com_df.reset_index(drop=True, inplace=True)

In [153]:
com_df['Filtered'].value_counts()

o    10789
Name: Filtered, dtype: int64

In [27]:
# 전체 논문에서 해당 년도 논문 수
df = pd.read_csv('전처리완료토큰.csv')
df.head()

,Year,tokens
0,2013,method peer peer streaming video demand reside...
1,2013,paper discus bacterial network 13_communicatio...
2,2013,article treat digital humanity work classical ...
3,2013,work describes preliminary step towards nano s...
4,2013,goal extraction learning 13_demonstration comp...


In [29]:
df = df[df['Year']>=2014]
a = df.groupby('Year')[['tokens']].count()
a.reset_index(drop=False, inplace=True)

In [157]:
a

,Year,tokens
0,2014,104942
1,2015,109697
2,2016,122464
3,2017,128825
4,2018,148154
5,2019,172502
6,2020,150604
7,2021,160111
8,2022,142047
9,2023,36158


In [31]:
import re

# 원하는 형식의 단어를 담을 정규표현식 패턴
pattern = r'\b\d{2}_\w+\b'  # 숫자 2개_단어 형식을 의미하는 패턴

# 결과를 저장할 리스트 초기화
filtered_sentences = []

# 각 문장에 대해 처리
for sentence in tqdm(df['tokens']):
    # 정규표현식을 사용하여 원하는 패턴에 맞는 단어들만 추출
    words = re.findall(pattern, sentence)

    # 추출한 단어들을 문장으로 다시 조합하여 결과 리스트에 추가
    filtered_sentence = ' '.join(words)
    filtered_sentences.append(filtered_sentence)
df['token2'] = filtered_sentences

100%|████████████████████████████| 1275504/1275504 [00:15<00:00, 81027.86it/s]


In [ ]:
# 전체 단어를 하나의 flatten 형태로 만들기

In [158]:
# 'Word' 칼럼과 '유사단어1'부터 '유사단어56'까지의 칼럼들에서 NaN이 아닌 값들 가져와서 리스트로 만들기
word_list = []
for index, row in com_df.iterrows():
    word = row['Word']
    similar_words = [row[f'유사단어{i}'] for i in range(1, 57) if not pd.isnull(row[f'유사단어{i}'])]
    word_list.extend([word] + similar_words)

# 리스트를 1차원 형태로 만들기
flattened_word_list = np.array(word_list).flatten().tolist()


In [159]:
flattened_word_list = list(set(flattened_word_list))

In [160]:
len(flattened_word_list)

32548

In [162]:
flattened_word_list

['18_location_privacy',
 '14_single_chip',
 '21_authoring_system',
 '15_modular_ontology',
 '18_generative_model',
 '14_code_smell',
 '18_burst',
 '17_activity_tracker',
 '19_capacitance',
 '19_cooling_system',
 '14_adequacy',
 '22_image_patch',
 '14_emergency_rescue',
 '17_alpha',
 '17_clinical_application',
 '18_carrier_synchronization',
 '18_unsupervised_deep_learning',
 '23_additive_manufacturing',
 '17_memory_performance',
 '15_dielectric_constant',
 '15_sign_language_translation',
 '15_water_distribution_system',
 '18_multiple_instance_learning',
 '21_middle_school',
 '17_computer_music',
 '23_power_distribution',
 '15_emergency_situation',
 '18_compass',
 '14_constraint_solving',
 '15_data_distribution',
 '22_background_knowledge',
 '19_duty_cycle',
 '22_appliance',
 '17_birefringence',
 '17_glaucoma',
 '16_coordinate_descent',
 '20_cat_swarm_optimization',
 '15_cross_disciplinary',
 '15_complex_system',
 '22_eye_detection',
 '22_denoise',
 '20_hbase',
 '15_iot_service',
 '16_in

In [161]:
int('20' + flattened_word_list[0][:2])

2018

In [139]:
word_count_list = []
for i in tqdm(range(50000,len(flattened_word_list))):
    find_word = flattened_word_list[i]  # 찾아야 할 단어 리스트
    year = int('20' + flattened_word_list[i][:2])
    find_df = df[df['Year'] == year]  # 찾아야 하는 연도의 초록들
    find_df.reset_index(drop=True, inplace=True)
    word_count = 0
    for j in range(len(find_df)):
        sentence = find_df['token2'][j]
        # 문장을 공백을 기준으로 분할하여 리스트로 변환
        words = sentence.split()

        # 하나의 단어라도 문장에 완전일치로 포함되는지 확인
        found = find_word in words
        if found:
            word_count += 1

    # 결과 출력
    word_count_list.append(word_count)

  0%|                                     | 12/8504 [00:06<1:11:29,  1.98it/s]


KeyboardInterrupt: 

In [163]:
years_dict = {}  # 연도별 초록들을 그룹화할 딕셔너리

# 연도별 초록들을 그룹화
for index, row in tqdm(df.iterrows()):
    year = row['Year']
    if year in years_dict:
        years_dict[year].append(row['token2'])
    else:
        years_dict[year] = [row['token2']]

1275504it [00:25, 50124.77it/s]


In [164]:
word_count_list = []

In [165]:
# 각 단어 리스트에 대해 검색
for find_word in tqdm(flattened_word_list):
    year = int('20' + find_word[:2])
    if year in years_dict:
        sentences = years_dict[year]
        word_set = set(find_word.split())
        word_count = sum(1 for sentence in sentences if any(word in word_set for word in sentence.split()))
    else:
        word_count = 0

    word_count_list.append(word_count)

100%|█████████████████████████████████| 32548/32548 [1:10:10<00:00,  7.73it/s]


In [167]:
len(word_count_list)

32548

In [174]:
word_entropy = []
for find_word in tqdm(flattened_word_list):
    year = int('20' + find_word[:2])
    if year == 2014:
        word_entropy.append(word_count_list[i]/a['tokens'][0])
    elif year == 2015:
        word_entropy.append(word_count_list[i]/a['tokens'][1])
    elif year == 2016:
        word_entropy.append(word_count_list[i]/a['tokens'][2])
    elif year == 2017:
        word_entropy.append(word_count_list[i]/a['tokens'][3])
    elif year == 2018:
        word_entropy.append(word_count_list[i]/a['tokens'][4])
    elif year == 2019:
        word_entropy.append(word_count_list[i]/a['tokens'][5])
    elif year == 2020:
        word_entropy.append(word_count_list[i]/a['tokens'][6])
    elif year == 2021:
        word_entropy.append(word_count_list[i]/a['tokens'][7])
    elif year == 2022:
        word_entropy.append(word_count_list[i]/a['tokens'][8])
    elif year == 2023:
        word_entropy.append(word_count_list[i]/a['tokens'][9])

100%|███████████████████████████████| 32548/32548 [00:00<00:00, 210988.99it/s]


In [173]:
find_word

78

In [175]:
word_entropy

[0.0002362406684935945,
 0.0003335175620819119,
 0.00021859834739649368,
 0.0003190606853423521,
 0.0002362406684935945,
 0.0003335175620819119,
 0.0002362406684935945,
 0.00027168639627401516,
 0.0002028961983049472,
 0.0002028961983049472,
 0.0003335175620819119,
 0.00024639731919716716,
 0.0003335175620819119,
 0.00027168639627401516,
 0.00027168639627401516,
 0.0002362406684935945,
 0.0002362406684935945,
 0.0009679738923613031,
 0.00027168639627401516,
 0.0003190606853423521,
 0.0003190606853423521,
 0.0003190606853423521,
 0.0002362406684935945,
 0.00021859834739649368,
 0.00027168639627401516,
 0.0009679738923613031,
 0.0003190606853423521,
 0.0002362406684935945,
 0.0003335175620819119,
 0.0003190606853423521,
 0.00024639731919716716,
 0.0002028961983049472,
 0.00024639731919716716,
 0.00027168639627401516,
 0.00027168639627401516,
 0.0002857982754115495,
 0.00023239754588191548,
 0.0003190606853423521,
 0.0003190606853423521,
 0.00024639731919716716,
 0.00024639731919716716,
 

In [176]:
ab = pd.DataFrame()
ab['word'] = flattened_word_list
ab['count'] = word_count_list
ab['entropy'] = word_entropy

In [178]:
ab.to_csv('단어카운트및엔트로피.csv', index=False, encoding='utf-8-sig')

In [181]:
import math

In [182]:
-(word_entropy[0]*math.log2(word_entropy[0]))

0.00284609883124353

In [183]:
word_entropy[0]*math.log2(word_entropy[0])

-0.00284609883124353

In [184]:
com_df

,Word,유사단어1,유사도1,유사단어2,유사도2,유사단어3,유사도3,유사단어4,유사도4,유사단어5,...,Similar_Word_Count,count,count_norm_paper,count_norm_paper_words,total_words,cite_count,cite_count_words_norm,count_norm_words,cite_count_norm,Filtered
0,19_machine_learning_approach,19_machine_learning_method,0.9038,19_machine_learning_technique,0.8505,19_classification_model,0.7503,19_classification_algorithm,0.7496,19_classification_technique,...,22,59712,0.007673,2596.173913,23,599807,26078.565220,0.000334,0.007421,o
1,20_computational_model,20_cognitive_model,0.6318,20_agent_based_model,0.6308,20_computational_tool,0.6200,20_computational_method,0.6146,20_cognitive_architecture,...,14,39444,0.005205,2629.600000,15,333511,22234.066670,0.000347,0.004702,o
2,21_machine_learning_approach,21_machine_learning_method,0.9089,21_machine_learning_technique,0.8917,21_classification_technique,0.7693,21_machine_learning_classifier,0.7501,21_classification_algorithm,...,21,58135,0.007416,2642.500000,22,593194,26963.363640,0.000337,0.007650,o
3,20_machine_learning_approach,20_machine_learning_method,0.8984,20_machine_learning_technique,0.8906,20_classification_technique,0.8130,20_deep_learning_architecture,0.7556,20_classification_model,...,29,57198,0.007547,1906.600000,30,482007,16066.900000,0.000252,0.006796,o
4,22_artificial_intelligence,22_ai_technology,0.7472,22_machine_learning,0.7247,22_iot_technology,0.6717,22_deep_learning,0.6655,22_internet_of_thing,...,40,40536,0.005170,988.682927,41,475747,11603.585370,0.000126,0.013100,o
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10784,23_homomorphic_encryption,23_digital_signature,0.6341,23_cryptographic,0.6061,23_cryptography,0.5878,23_encrypted_data,0.5802,23_encryption_scheme,...,5,545,0.000603,90.833333,6,780,130.000000,0.000101,0.000036,o
10785,23_construction_industry,23_construction_project,0.6488,23_advanced_technology,0.6185,23_building_information_modeling,0.5623,23_healthcare_system,0.5441,23_continuous_improvement,...,5,273,0.000302,45.500000,6,753,125.500000,0.000050,0.000035,o
10786,23_community_detection,23_community_discovery,0.6050,23_unsupervised_machine_learning,0.5934,23_community_structure,0.5751,23_ant_colony_optimization,0.5536,23_image_dehazing,...,5,147,0.000163,24.500000,6,620,103.333333,0.000027,0.000029,o
10787,23_action_recognition,23_object_detection,0.7092,23_image_classification,0.6398,23_face_detection,0.6169,23_deep_learning,0.6118,23_depth_estimation,...,7,3796,0.004201,474.500000,8,608,76.000000,0.000525,0.000028,o


In [191]:
import math

def shannon_entropy(probabilities):
    entropy = 0
    for prob in probabilities:
        if prob > 0:
            entropy -= prob * math.log2(prob)
    return entropy

# 예시: 주사위를 던졌을 때 각 눈이 나올 확률
probabilities = [1/6, 1/6, 1/6, 1/6, 1/6, 1/6]
result_entropy = shannon_entropy(probabilities)
print("Shannon Entropy:", result_entropy)

Shannon Entropy: 2.584962500721156


In [196]:
# 데이터프레임에서 'Word' 칼럼과 '유사도1', '유사도2', ..., '유사도56'까지의 칼럼들에서 NaN이 아닌 값을 가져와서 리스트로 저장
word_and_similar_list = []

for index, row in com_df.iterrows():
    word = row['Word']
    similar_words = [row[f'유사단어{i}'] for i in range(1, 57) if pd.notnull(row[f'유사도{i}'])]
    word_and_similar_list.append([word] + similar_words)

In [199]:
ab

,word,count,entropy
0,18_location_privacy,78,0.000236
1,14_single_chip,67,0.000334
2,21_authoring_system,10,0.000219
3,15_modular_ontology,5,0.000319
4,18_generative_model,329,0.000236
...,...,...,...
32543,16_air_quality_monitoring,24,0.000286
32544,17_high_speed_network,40,0.000272
32545,20_motion_recognition,24,0.000232
32546,16_handwritten_character_recognition,21,0.000286


In [208]:
total_entropy = []
for i in tqdm(range(len(word_and_similar_list[:50]))):
    entropy_list = []
    for j in range(len(word_and_similar_list[i])):
        for k in range(len(ab)):
            if word_and_similar_list[i][j]==ab['word'][k]:
                entropy_list.append(ab['entropy'][k])
    result_entropy = shannon_entropy(entropy_list)
    total_entropy.append(result_entropy)

100%|█████████████████████████████████████████| 50/50 [02:26<00:00,  2.93s/it]


In [203]:
len(entropy_list)

23

In [204]:
len(word_and_similar_list[0])

23

In [209]:
total_entropy

[0.057245198836411715,
 0.04207947140469546,
 0.05847668812049228,
 0.08415894280939094,
 0.12109338142750613,
 0.06261417428735766,
 0.03189637533845034,
 0.0386200149116458,
 0.07176684451151327,
 0.08239896962433008,
 0.12700037564348204,
 0.12109338142750613,
 0.09538013518397641,
 0.06830637194984471,
 0.06910881323330907,
 0.06436669151940967,
 0.036468875217402726,
 0.07217872896764953,
 0.06436669151940967,
 0.12404687853549408,
 0.08367669897523257,
 0.12404687853549408,
 0.05187746200117802,
 0.04207947140469546,
 0.08771503218073848,
 0.12995387275147,
 0.13290736985945795,
 0.10940662565220824,
 0.08689503355120305,
 0.053160625564083894,
 0.07466765065618916,
 0.027378138573936037,
 0.1358608669674459,
 0.10100518857175948,
 0.08505700090253428,
 0.07781619300176701,
 0.16834933515531336,
 0.07781619300176701,
 0.12109338142750613,
 0.1037960294649155,
 0.07704222097499916,
 0.09257483709033004,
 0.044884769498341824,
 0.044884769498341824,
 0.05581865684228809,
 0.1270003

In [213]:
total_entropy = []
word_entropy_mapping = {}  # 단어와 엔트로피를 매핑한 딕셔너리

# ab 데이터프레임에서 단어와 엔트로피를 딕셔너리로 매핑
for index, row in ab.iterrows():
    word_entropy_mapping[row['word']] = row['entropy']

# word_and_similar_list의 각 단어에 대해 엔트로피를 계산
for i in tqdm(range(len(word_and_similar_list))):
    entropy_list = []
    for word in word_and_similar_list[i]:
        entropy = word_entropy_mapping.get(word)
        if entropy is not None:
            entropy_list.append(entropy)
    result_entropy = shannon_entropy(entropy_list)
    total_entropy.append(result_entropy)

100%|███████████████████████████████| 10789/10789 [00:00<00:00, 119783.33it/s]


In [214]:
len(total_entropy)

10789

In [215]:
len(com_df)

10789

In [217]:
com_df['entropy'] = total_entropy

In [218]:
com_df.drop('Similar_Word_Count', axis=1, inplace=True)

In [220]:
com_df.to_csv('엔트로피반영군집데이터.csv', index=False, encoding='utf-8-sig')